### INGESTION

In [1]:
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime

In [2]:
#############Load config.json and get input and output paths
with open('config.json','r') as f:
    config = json.load(f) 

input_folder_path = config['input_folder_path']
print(input_folder_path)
output_folder_path = config['output_folder_path']
print(output_folder_path)

practicedata
ingesteddata


In [19]:
main_df = pd.DataFrame(columns=[
    'corporation', 'lastmonth_activity', 'lastyear_activity', 'number_of_employees', 'exited'
    ])
main_df.head(1)

,corporation,lastmonth_activity,lastyear_activity,number_of_employees,exited


In [27]:
ingested_files = []

In [28]:
current_dir = os.getcwd()
input_dir = os.path.join(os.getcwd(), input_folder_path)
input_files = os.listdir(input_dir)
for file_name in input_files:
    print(file_name)
    if '.csv' in file_name:
        print(os.path.join(input_dir,file_name))
        df = pd.read_csv(os.path.join(input_dir,file_name))
        main_df = pd.concat([main_df, df], ignore_index=True)
        ingested_files.append(file_name)

dataset2.csv
\\wsl.localhost\Ubuntu\home\chafund\GIT\Dynamic_Risk_Assessment_System\practicedata\dataset2.csv
dataset1.csv
\\wsl.localhost\Ubuntu\home\chafund\GIT\Dynamic_Risk_Assessment_System\practicedata\dataset1.csv


In [29]:
ingested_files

['dataset2.csv', 'dataset1.csv']

In [22]:
main_df.head()

,corporation,lastmonth_activity,lastyear_activity,number_of_employees,exited
0,abcd,78,1024,12,1
1,asdf,14,2145,20,0
2,xyzz,182,3891,35,0
3,acme,101,10983,2,1
4,qwer,0,118,42,1


In [23]:
print(main_df.shape)
main_df = main_df.drop_duplicates()
print(main_df.shape)

(36, 5)
(26, 5)


In [24]:
# save to outout folder
output_dir = os.path.join(os.getcwd(), output_folder_path)
print(output_dir)

\\wsl.localhost\Ubuntu\home\chafund\GIT\Dynamic_Risk_Assessment_System\ingesteddata


In [26]:
main_df.to_csv(os.path.join(output_dir, 'finaldata.csv'))

In [30]:
with open(os.path.join(output_dir, 'ingestedfiles.txt'), 'w') as tfile:
    tfile.write('\n'.join(ingested_files))

### MODEL TRAINING

In [1]:
from flask import Flask, session, jsonify, request
import pandas as pd
import numpy as np
import pickle
import os
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import json

In [2]:
# Load config.json and get path variables
with open('config.json','r') as f:
    config = json.load(f) 

dataset_csv_path = os.path.join(config['output_folder_path'])
print(dataset_csv_path)
model_path = os.path.join(config['output_model_path'])
print(model_path)

ingesteddata
practicemodels


In [3]:
data_dir = os.path.join(os.getcwd(), dataset_csv_path)
print(data_dir)

\\wsl.localhost\Ubuntu\home\chafund\GIT\Dynamic_Risk_Assessment_System\ingesteddata


In [8]:
data = pd.read_csv(os.path.join(data_dir, 'finaldata.csv'), index_col=0)
print(data.shape)
data.head()

(26, 5)


,corporation,lastmonth_activity,lastyear_activity,number_of_employees,exited
0,abcd,78,1024,12,1
1,asdf,14,2145,20,0
2,xyzz,182,3891,35,0
3,acme,101,10983,2,1
4,qwer,0,118,42,1


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 0 to 29
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   corporation          26 non-null     object
 1   lastmonth_activity   26 non-null     int64 
 2   lastyear_activity    26 non-null     int64 
 3   number_of_employees  26 non-null     int64 
 4   exited               26 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 1.2+ KB


In [24]:
X = data[['lastmonth_activity', 'lastyear_activity', 'number_of_employees']]
y = data['exited']
print(y.shape)

(26,)


In [29]:
lg = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                intercept_scaling=1, l1_ratio=None, max_iter=100,
                multi_class='auto', n_jobs=None, penalty='l2',
                random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                warm_start=False)

model = lg.fit(X, y)

In [30]:
print(os.path.join(os.getcwd(), model_path, 'trainedmodel.pkl'))

\\wsl.localhost\Ubuntu\home\chafund\GIT\Dynamic_Risk_Assessment_System\practicemodels\trainedmodel.pkl


In [31]:
pickle.dump(model, open(os.path.join(os.getcwd(), model_path, 'trainedmodel.pkl'), 'wb'))

### MODEL SCORING

In [32]:
from flask import Flask, session, jsonify, request
import pandas as pd
import numpy as np
import pickle
import os
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import json

In [36]:
# Load config.json and get path variables
with open('config.json','r') as f:
    config = json.load(f) 

model_path = os.path.join(config['output_model_path'])
print(model_path)
test_data_path = os.path.join(config['test_data_path'])
print(test_data_path)

practicemodels
testdata


In [37]:
model_dir = os.path.join(os.getcwd(), model_path)
print(model_dir)

\\wsl.localhost\Ubuntu\home\chafund\GIT\Dynamic_Risk_Assessment_System\practicemodels


In [42]:
model = pickle.load(open(os.path.join(os.getcwd(), model_dir, 'trainedmodel.pkl'), 'rb'))
print(type(model))

<class 'sklearn.linear_model._logistic.LogisticRegression'>


In [44]:
testdata_dir = os.path.join(os.getcwd(), test_data_path)
testdata = pd.read_csv(os.path.join(testdata_dir, 'testdata.csv'))
print(testdata.shape)

(5, 5)


In [45]:
X_test = testdata[['lastmonth_activity', 'lastyear_activity', 'number_of_employees']]
y_test = testdata['exited']

In [46]:
pred = model.predict(X_test)

In [48]:
f1score = metrics.f1_score(pred, y_test)

In [51]:
with open(os.path.join(model_path, 'latestscore.txt'), 'w') as tfile:
        tfile.write(str(f1score))

### MODEL DEPLOYMENT

In [52]:
from flask import Flask, session, jsonify, request
import pandas as pd
import numpy as np
import pickle
import os
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import json

In [63]:
with open('config.json','r') as f:
    config = json.load(f)

model_path = os.path.join(config['output_model_path'])
print(model_path)
ingested_data_path = os.path.join(config['output_folder_path'])
print(ingested_data_path)
prod_deployment_path = os.path.join(config['prod_deployment_path'])
print(prod_deployment_path)

practicemodels
ingesteddata
production_deployment


In [59]:
latestscore_dir = os.path.join(os.getcwd(), model_path)
latestscore = open(os.path.join(os.getcwd(), latestscore_dir, 'latestscore.txt'), 'r').read()

output_dir = latestscore_dir = os.path.join(os.getcwd(), prod_deployment_path)
with open(os.path.join(os.getcwd(), output_dir, 'latestscore.txt'), 'w') as txt_file:
    txt_file.write(str(latestscore))

In [62]:
ingested_dir = os.path.join(os.getcwd(), ingested_data_path)
ingested_data = open(os.path.join(os.getcwd(), ingested_dir, 'ingestedfiles.txt'), 'r').read()
print(ingested_data)

with open(os.path.join(os.getcwd(), output_dir, 'ingestedfiles.txt'), 'w') as txt_file:
    txt_file.write(str(ingested_data))

dataset2.csv
dataset1.csv


In [66]:
model_dir = os.path.join(os.getcwd(), model_path)
print(model_dir)

\\wsl.localhost\Ubuntu\home\chafund\GIT\Dynamic_Risk_Assessment_System\practicemodels


In [69]:
import shutil

In [70]:
shutil.copyfile(
    os.path.join(os.getcwd(), model_dir, 'trainedmodel.pkl'),
    os.path.join(os.getcwd(), output_dir, 'trainedmodel.pkl')
)

'\\\\wsl.localhost\\Ubuntu\\home\\chafund\\GIT\\Dynamic_Risk_Assessment_System\\production_deployment\\trainedmodel.pkl'